# Epipolar Geometry and Eight Point Algorithm (20 Points)

## Epipolar Geometry

In [ ]:
import numpy as np
import pyvista as pv
import trimesh
from utils.utils import (
    load_scene_params,
    load_scene_params2,
    plot_epipolar_setup,
    draw_dot,
    draw_line,
    superglue_correspondences,
    draw_matches,
    draw_candidates,
    draw_pair_with_reconstruction,
    test_normalize_points
)
import matplotlib.pyplot as plt
import cv2

We have prepared a couple of scenes containing some object which is viewed from two different angles. Load scene parameters, 3D mesh and images using `load_scene_params`. `scene_params` contains the camera intrinsic and extrinsic parameters, as well as the projection matrices and global positions of the camera centers.

In [ ]:
scene_params, mesh, img0, img1 = load_scene_params(scene_id="01")
print(scene_params.keys())

In [ ]:
# Display the two camera views side by side
plt.figure(figsize=(12, 6))

# Plot first image
plt.subplot(1, 2, 1)
plt.imshow(img0)
plt.title("Camera 0")
plt.axis('off')

# Plot second image
plt.subplot(1, 2, 2)
plt.imshow(img1)
plt.title("Camera 1")
plt.axis('off')

plt.tight_layout()
plt.show()

You can also visualize the setting in 3D using `plot_epipolar_setup`. You should get an interactive 3D plot of the scene with the camera centers, camera frustrums, camera coordinate systems, and 3D mesh.

In [ ]:
pl = plot_epipolar_setup(mesh, img0, img1, scene_params)
pl.show()

# Epipolar Geometry

### Essential Matrix and Fundamental Matrix Computation (2 Points)
Implement the function `compute_E_F` that given the scene params, computes the essential matrix $E$ and fundamental matrix $F$ from the camera intrinsics and the relative camera pose.

In [ ]:
def compute_E_F(scene_params):
    """
    Takes as input the scene parameters dictionary and returns (E, F) the essential and fundamental matrix.
    """
    # TODO: Implement your code here

In [ ]:
E, F = compute_E_F(scene_params)
print("Essential Matrix:\n", E)
print("Fundamental Matrix:\n", F)

Verify the epipolar constraint by taking a random 3D mesh point, computing the projections to the image planes and evaluating $u_0^T F u_1$. The results should be close to zero.

In [ ]:
point_ids = np.random.randint(0, len(mesh.vertices), 10)
p3d = mesh.vertices[point_ids]

## Verify the epipolar constraint using the points above.
## TODO: Implement your code here

### Epipoles (1 Points)
Recall that the epipoles are the image points where the baseline intersects the image planes. In other words, the epipoles are the projections of the camera centers in the other image. From the lecture you know that the epipoles can be computed from the fundamental matrix. Find the epipoles in both images and plot them on the images using the function `draw_dot`.

In [ ]:
## Find the epipoles e0 and e1 in image coordinates.
## TODO: Implement your code here
e0 = ...
e1 = ...

In [ ]:
# Draw epipoles in green in images.
img0_ep = img0.copy()
img1_ep = img1.copy()
# Draw epipoles in images
img0_ep = draw_dot(img0_ep, e0, color=(0, 255, 0, 255), radius=7)
img1_ep = draw_dot(img1_ep, e1, color=(0, 255, 0, 255), radius=7)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img0_ep)
plt.title("Epipole in image 0")
plt.subplot(1, 2, 2)
plt.imshow(img1_ep)
plt.title("Epipole in image 1")
plt.show()

### Epipolar Lines (2 Points)

Given a 3D point, we want to find the epipolar lines in the two images.

In [ ]:
PID = 100 # Point ID. Try: 354 (nose), 21926 (medal), 20405 (hair)
point = mesh.vertices[PID]

# Find the epipolar lines in both images for the 3D point point.
# TODO: Implement your code here
l0 = ...
l1 = ...

# Draw epipolar lines in images
img0_line = img0.copy()
img1_line = img1.copy()
img0_line = draw_line(img0_line, l0, color=(255, 0, 0, 255))
img1_line = draw_line(img1_line, l1, color=(255, 0, 0, 255))
img0_line = draw_dot(img0_line, e0, color=(0, 255, 0, 255), radius=7)
img1_line = draw_dot(img1_line, e1, color=(0, 255, 0, 255), radius=7)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img0_line)
plt.title("Epipolar line in image 0")
plt.subplot(1, 2, 2)
plt.imshow(img1_line)
plt.title("Epipolar line in image 1")
plt.show()

# Estimating Relative Pose of Two Cameras From 2D-2D Correspondences

In this exercise we will implement the normalized 8-point algorithm to estimate the fundamental matrix from 2D-2D point correspondences. Then we will decompose the essential matrix to estimate the relative pose of the two cameras. Finally, we will triangulate the 3D points from the 2D-2D correspondences and the estimated relative pose.

## Correspondences
We will use SuperPoint and SuperGlue for feature extraction and matching. Note that these correspondence matching algorithms are not perfect and will produce outliers. So if you observe inaccurate results, it can be due to bad correspondences. Below we provide a quick and dirty way of computing perfect correspondences via projection. We advise you to use this for debugging your code, to eliminate the effect of bad correspondences. 

In [ ]:
# Feel free to try other scenes (not "01", because there are not enough matches)
scene_params, img0, img1 = load_scene_params2(scene_id="06")

In [ ]:
# Display the two camera views side by side
plt.figure(figsize=(12, 6))

# Plot first image
plt.subplot(1, 2, 1)
plt.imshow(img0)
plt.title("Camera 0")
plt.axis('off')

# Plot second image
plt.subplot(1, 2, 2)
plt.imshow(img1)
plt.title("Camera 1")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
points0, points1 = superglue_correspondences(img0, img1, kth=0.001, mth=0.5)

In [ ]:
##############################
# Uncomment the following codeblock if you want to use perfect correspondences, computed from the ground truth 3D 
# mesh, instead of the correspondences found by superpoint and superglue. 
# This includes also matches which cannot be visible on both images, as they are on the back side of the object etc.
# You can use this to test your reconstruction code.
##############################

# SAMPLES = 10000
# p3d = np.random.randint(0, len(mesh.vertices), SAMPLES)
# p3d = mesh.vertices[p3d]
# p3d_hom = np.hstack([p3d, np.ones((SAMPLES, 1))])
# P0 = scene_params["P0"] # camera 0 projection matrix
# P1 = scene_params["P1"] # camera 1 projection matrix
# p2d_hom0 = p3d_hom @ P0.T
# p2d_hom1 = p3d_hom @ P1.T
# points0 = p2d_hom0[:, :2] / p2d_hom0[:, 2][:, None]
# points1 = p2d_hom1[:, :2] / p2d_hom1[:, 2][:, None]

Let us visualize the 2D-2D correspondences below.

In [ ]:
matched_image = draw_matches(img0, img1, points0, points1)
plt.figure(figsize=(10, 5))
plt.imshow(matched_image)
plt.title("Correspondences")
plt.axis("off")
plt.show()

## Normalized 8-Point Algorithm (10 Points)

Next we implement the normalized 8-point algorithm to estimate the fundamental matrix from the 2D-2D correspondences.
For this we first have to normalize the points such that their centroid (mean) is at the origin and the average distance to the origin is around $\sqrt{2}$.
Implement the function `normalize_points` that takes a set of 2D points and returns the normalized points and the transformation matrix $\mathbf{T}$ that was used to normalize the points.

In [ ]:
def normalize_points(points):
    """
    Normalize 2D points for improved numerical stability.
    
    Parameters:
        points (np.ndarray): 2D points, shape (n, 2)
        
    Returns:
        normalized_points (np.ndarray): Normalized points, shape (n, 2)
        T (np.ndarray): Transformation matrix, shape (3, 3)
    """
    # TODO: Implement this function

You can test your function by calling the function `test_normalize_points`. It will show the original 2D image points and the normalized image points next to each other. The normalized points should be centered around the origin and have an average distance of $\sqrt{2}$, while keeping the same relative structure.

In [ ]:
norm_points0, T0 = normalize_points(points0)
norm_points1, T1 = normalize_points(points1)

test_normalize_points(points0, norm_points0)
test_normalize_points(points1, norm_points1)

Next, implement the normalized 8-point algorithm in the function `eight_point`. The function should take the original image points, and return the estimated fundamental matrix $\mathbf{F}$. Make sure to make use of the normalization function you implemented before. Also make sure to enforce the rank-2 constraint on the fundamental matrix.

In [ ]:
def eight_point(p0, p1):
    """
    Normalized 8-point algorithm for fundamental matrix estimation.
    
    Parameters:
        p0 (np.ndarray): Points in image 0, shape (n, 2) where n >= 8
        p1 (np.ndarray): Corresponding points in image 1, shape (n, 2) where n >= 8
        
    Returns:
        F (np.ndarray): Fundamental matrix of shape (3, 3)
    """
    
    # TODO: Implement this function
    
    

F = eight_point(points0, points1)
print(F)

Let us estimate the fundamental matrix from the correspondences, and compare the epipoles and epipolar lines with the ground-truth values.

In [ ]:
def draw_epipolar_lines(img0, points0, img1, points1, F):
    epiline_img0 = img0.copy()
    epiline_img1 = img1.copy()
    U, S, Vt = np.linalg.svd(F)
    epi0 = U[:, 2]
    epi0 /= epi0[2]
    epi1 = Vt[-1]
    epi1 /= epi1[2]
    for i in range(len(points0)):
        l0 = F @ np.hstack([points1[i], 1])
        l1 = F.T @ np.hstack([points0[i], 1])
        epiline_img0 = draw_line(epiline_img0, l0, color=(255, 0, 0, 255), width=2)
        epiline_img1 = draw_line(epiline_img1, l1, color=(255, 0, 0, 255), width=2)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(epiline_img0)
    plt.title("Epipolar line in image 0")
    plt.subplot(1, 2, 2)
    plt.imshow(epiline_img1)
    plt.title("Epipolar line in image 1")
    plt.show()

draw_epipolar_lines(img0, points0[:10], img1, points1[:10], F)

# Ground truth comparison
_, F_gt = compute_E_F(scene_params)
draw_epipolar_lines(img0, points0[:10], img1, points1[:10], F_gt)

What are the estimated epipoles? What are the real epipoles?
- Estimated: 
- Real:

In [ ]:
# TODO: Compute epipoles here

Finally, compute the estimate for the essential matrix $\mathbf{E}$ from the fundamental matrix $\mathbf{F}$ and the camera intrinsics $\mathbf{K}_1$ and $\mathbf{K}_2$. The $\mathbf{K}_i$ are part of the scene parameters.

In [ ]:
# TODO: Compute essential matrix
E = ...

## Recovering Relative Pose (5 Points)
From the lecture you know that from the essential matrix $E$ we can recover the relative pose of the cameras. There are four possible solutions for the relative pose, but in only one of them, the scene is in front of both cameras. Implement the function `recover_relative_pose` that takes the essential matrix and returns the four possible relative poses.

In [ ]:
def pose_candidates(E):
    """
    Compute the four possible camera poses from the essential matrix.
    
    Parameters:
        E (np.ndarray): Essential matrix of shape (3, 3)
        
    Returns:
        Candidates (list): List of 4 possible camera poses, each represented as a list [R, t]
    """
    
    # TODO: Implement this function
    

candidates = pose_candidates(E)

We can visualize the relative poses by drawing the camera frustums using `draw_candidates`. Note that each frustum has a little triangle on top pointing upwards in image frame! One of the candidates should be oriented corretly relatively to the camera 0.
Hint for the viewer: You can use Ctrl + Left Click to rotate the viewer's camera around it's viewing axis.

In [ ]:
draw_candidates(candidates)

We know that that the correct relative pose is among these four candidates. To find the best one, we can triangulate the 3D points from the 2D-2D correspondences, and check which configuration gives us the most points in front of both cameras. Implement the function `find_best_candidate` that takes the candidates, 2D-2D correspondences and the scene params as input and returns the best candidate, as well as the triangulated 3D points. 
You can use `cv2.triangulatePoints` for this.

In [ ]:
def find_best_candidate(candidates, points0, points1, scene_params):
    """
    Find the best candidate pose by triangulating points and counting the number of points in front of both cameras.
    Parameters:
        candidates (list): List of candidate poses
        points0 (np.ndarray): Points in image 0, shape (n, 2)
        points1 (np.ndarray): Points in image 1, shape (n, 2)
        scene_params (dict): Scene parameters including camera intrinsics and extrinsics
    Returns:
        best_candidate (tuple): Best candidate pose (R, t)
        estimated_point_cloud (np.ndarray): Estimated 3D point cloud, shape (n, 3)
    """
    
    # TODO: Implement this function
    

best_candidate, estimated_point_cloud = find_best_candidate(candidates, points0, points1, scene_params)    

Finally, let us plot the estimated relative camera poses together with the triangulated 3D points. Use the function `draw_pair_with_reconstruction` which takes as input:
- $\mathbf{K}_0$,
- $\mathbf{K}_1$,
- the extrinsic parameters of camera 1 (recall that we assume that camera 0's frame = world frame),
- the two images (for sizes of image planes and colors),
- 2D points in image 0 and 1, to extract color information,
- the triangulated 3D points.

You only have to pass the correct arguments to the function. The function will plot the two images with the triangulated 3D points and the camera frustums.

In [ ]:
# TODO: Compute the extrinsic matrix RT
RT = ...

draw_pair_with_reconstruction(scene_params["K0"], 
                              scene_params["K1"], 
                              RT, 
                              img0, 
                              img1, 
                              points0, 
                              points1, 
                              estimated_point_cloud, 
                              size=15)